In [27]:
#%pylab inline

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pandas
import math
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.regularizers import l1l2
import cPickle as pickle

In [11]:
matrix = pandas.read_csv('/home/rbhat/Workspace/ConesaGenomics/NEW_matrix.csv')
#plt.plot(dataframe)
#plt.show()

In [12]:
#matrix.head()

In [13]:
gene_exp_TP_matrix = []
gene_exp_TP_matrix = list(matrix[matrix['Label'] == 'True_POSITIVE']['GENE_EXP'])
#print gene_exp_TP_matrix

gene_exp_matrix = []
gene_exp_TN_matrix = list(matrix[matrix['Label'] == 'True_NEGATIVE']['GENE_EXP'])
#print gene_exp_TN_matrix

In [14]:
gene_exp_TP_matrix.extend(gene_exp_TN_matrix)

In [15]:
Y1= np.ones(36)
Y2 = np.zeros(22)

Y = np.insert(Y1, 36, Y2)

In [16]:
X_input=[]

for i in range(len(gene_exp_TP_matrix)):
    temp=[]
    temp.append(gene_exp_TP_matrix[i])
    X_input.append(temp)

In [17]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
enc.fit(X_input)
X = enc.transform(X_input).toarray()

X = np.expand_dims(X, axis=1)

In [18]:
#give labels

#Y = np.expand_dims(Y, axis=1)
print Y.shape
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, np.asarray(Y), test_size=0.20, random_state=42)

(58,)


In [19]:
print ('shape: ',X_train.shape)
print ('shape: ',y_train.shape)
print ('shape: ',X_test.shape)
print ('shape: ',y_test.shape)

('shape: ', (46, 1, 57))
('shape: ', (46,))
('shape: ', (12, 1, 57))
('shape: ', (12,))


In [20]:
import numpy as np
#import tensorflow as tf
#tf.python.control_flow_ops = tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, Convolution1D, MaxPooling1D
from keras.utils import np_utils
#from keras import backend as K

batch_size = 8
nb_classes = 2
nb_epoch = 10

model = Sequential()
model.add(Convolution1D(16, 3, border_mode='same', input_shape=(1,57)))
model.add(Activation('relu'))
model.add(Convolution1D(32, 3, border_mode='same'))
model.add(Activation('relu'))

model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, y_test))
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Using Theano backend.
Using gpu device 0: Tesla M40 (CNMeM is disabled, cuDNN 5005)


Exception: Error when checking model target: expected activation_4 to have shape (None, 2) but got array with shape (46, 1)

In [23]:
kfold = model_selection.KFold(n_splits=10, random_state=1234)

In [24]:
estimators = []
model1 = LogisticRegression()
estimators.append(('logistic', model1))
model2 = DecisionTreeClassifier()
estimators.append(('cart', model2))

In [25]:
# create the ensemble model
ensemble = VotingClassifier(estimators)
results = model_selection.cross_val_score(ensemble, X_train.squeeze(), y_train, cv=kfold)
print(results.mean())

0.67


In [30]:
def model_factory(n_classes, n_dims):
    print("Building model...")

    lmbd1 = 0
    lmbd2 = 0
    
    model = Sequential()
    model.add(Convolution1D(16, 3, border_mode='same', input_shape=(1,57)))
    model.add(Activation('relu'))
    model.add(Convolution1D(32, 3, border_mode='same'))
    model.add(Activation('relu'))

    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(16))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])

    return model

In [36]:
n_models = 20
print("Training %d models..." % n_models)

proba = 0
models = range(1, n_models+1)
for i in models:
    print("\n-------------- Model %d --------------\n" % i)
    model = model_factory(n_classes, n_dims)
    model.fit(X, y, nb_epoch=120, batch_size=128, validation_split=0.0,
              show_accuracy=True, verbose=2)
    proba += model.predict_proba(X_test, verbose=2)
    with open(os.path.join(paths.model_path, model_name+'.pkl'), 'wb') as f:
        pickle.dump((proba, i), f, protocol=pickle.HIGHEST_PROTOCOL)

Training 20 models...

-------------- Model 1 --------------



NameError: name 'n_classes' is not defined